In [3]:
import json
import csv
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import Point, MultiPoint, Polygon, MultiPolygon
from shapely.affinity import scale
import matplotlib.pyplot as plt

import glob
import os
import datetime

# Weather_Assumptions_thru_2016-2019

In [4]:
#loading all data and concatenating
path = r'/Users/margaretmccall/Documents/2020 Spring/CE 295/0 - Final Project/Data--ERCOT/Weather_Assumptions_thru_2016-2019'
all_files = glob.glob(path + "/*.csv")

dfs = []
bad_files = []

for file in all_files:
    try:
        x = pd.read_csv(file)
        dfs.append(x)
    except pd.errors.EmptyDataError:
        bad_files.append(file)
        
df = pd.concat(dfs, ignore_index=True)

In [5]:
df.head()
#TODO: HOUR_ENDING RUNS FROM 0:23 SO I CANT JUST SUBTRACT

,DeliveryDate,HourEnding,Coast,East,FarWest,North,NorthCentral,SouthCentral,Southern,West,DSTFlag
0,02/19/2016,01:00,63.1,61.5,61.0,64.5,65.00,63.5,66.0,66.2,N
1,02/19/2016,02:00,63.6,61.0,60.0,63.0,63.75,64.0,64.8,64.2,N
2,02/19/2016,03:00,63.4,61.5,59.0,63.0,63.25,63.5,64.0,63.0,N
3,02/19/2016,04:00,63.7,62.0,59.0,62.5,62.50,64.0,63.2,62.4,N
4,02/19/2016,05:00,62.6,62.0,54.0,60.5,61.25,64.0,62.0,61.8,N


In [6]:
df['hr_end'] = df['HourEnding'].apply(lambda x: int(x[:2]))
df['hr_beg'] = df['hr_end'] - 1
df.drop(columns=['hr_end'],inplace=True)

In [8]:
max(df['HourEnding'])

'23:00'

In [11]:
dupes = df[df['DSTFlag']=='Y'] #i now remember from the old days of parsing ERCOT data
#that there's always some weird thing at 3pm with duplicate data

In [15]:
dupe_avgs = dupes.groupby(by=['DeliveryDate','HourEnding']).mean()

In [2]:
dupe_avgs

NameError: name 'dupe_avgs' is not defined

In [39]:
df.drop_duplicates(subset=['DeliveryDate','HourEnding'], keep="first", inplace=True)

In [43]:
df.reset_index(inplace=True)

In [45]:
df.rename(columns={'DeliveryDate': 'date'}, inplace=True)

In [47]:
df.drop(columns=['index','HourEnding'], inplace=True)

In [49]:
df.drop(columns=['DSTFlag'], inplace=True)

In [50]:
df.to_csv('weather_forecast_ercot.csv', index=False)

# WPP_Hrly_AVG_Actual_Forecasted_2016-2018

In [63]:
#loading all data and concatenating
path = r'/Users/margaretmccall/Documents/2020 Spring/CE 295/0 - Final Project/Data--ERCOT/WPP_Hrly_AVG_Actual_Forecasted_2016-2018'
all_files = glob.glob(path + "/*.csv")

dfs = []
bad_files = []

for file in all_files:
    try:
        x = pd.read_csv(file)
        dfs.append(x)
    except pd.errors.EmptyDataError:
        bad_files.append(file)
        
df_wind = pd.concat(dfs, ignore_index=True)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [64]:
df_wind.head()

,ACTUAL_LZ_NORTH,ACTUAL_LZ_SOUTH_HOUSTON,ACTUAL_LZ_WEST,ACTUAL_SOUTH_HOUSTON,ACTUAL_SYSTEM_WIDE,ACTUAL_WEST_NORTH,COP_HSL_LZ_NORTH,COP_HSL_LZ_SOUTH_HOUSTON,COP_HSL_LZ_WEST,COP_HSL_SOUTH_HOUSTON,COP_HSL_SYSTEM_WIDE,COP_HSL_WEST_NORTH,DELIVERY_DATE,DSTFlag,HOUR_BEGINNING,HOUR_ENDING,STWPF_LZ_NORTH,STWPF_LZ_SOUTH_HOUSTON,STWPF_LZ_WEST,STWPF_SOUTH_HOUSTON,STWPF_SYSTEM_WIDE,STWPF_WEST_NORTH,WGRPP_LZ_NORTH,WGRPP_LZ_SOUTH_HOUSTON,WGRPP_LZ_WEST,WGRPP_SOUTH_HOUSTON,WGRPP_SYSTEM_WIDE,WGRPP_WEST_NORTH
0,686.95,2899.57,9815.77,NaN,13402.29,NaN,737.5,2723.7,11641.4,NaN,15102.6,NaN,03/22/2018,N,NaN,5.0,804.1,2768.1,11941.3,NaN,15513.5,NaN,726.3,2350.5,10744.3,NaN,13821.1,NaN
1,653.45,2776.28,10543.93,NaN,13973.66,NaN,756.8,2650.5,11376.6,NaN,14783.9,NaN,03/22/2018,N,NaN,6.0,798.0,2851.6,11866.8,NaN,15516.4,NaN,720.2,2434.0,10670.0,NaN,13824.2,NaN
2,671.39,2712.13,11113.10,NaN,14496.62,NaN,757.0,2643.5,11265.7,NaN,14666.2,NaN,03/22/2018,N,NaN,7.0,778.7,2747.0,11580.0,NaN,15105.7,NaN,700.7,2329.8,10386.3,NaN,13416.8,NaN
3,638.67,2643.54,11065.38,NaN,14347.59,NaN,760.0,2740.3,10913.0,NaN,14413.3,NaN,03/22/2018,N,NaN,8.0,753.8,2686.8,11309.0,NaN,14749.6,NaN,675.9,2269.3,10113.9,NaN,13059.1,NaN
4,706.22,2635.47,10526.52,NaN,13868.21,NaN,720.9,2788.7,10346.3,NaN,13855.9,NaN,03/22/2018,N,NaN,9.0,765.4,2746.9,10817.3,NaN,14329.6,NaN,687.8,2329.6,9622.5,NaN,12639.9,NaN


In [65]:
df_wind.columns

Index(['ACTUAL_LZ_NORTH', 'ACTUAL_LZ_SOUTH_HOUSTON', 'ACTUAL_LZ_WEST',
       'ACTUAL_SOUTH_HOUSTON', 'ACTUAL_SYSTEM_WIDE', 'ACTUAL_WEST_NORTH',
       'COP_HSL_LZ_NORTH', 'COP_HSL_LZ_SOUTH_HOUSTON', 'COP_HSL_LZ_WEST',
       'COP_HSL_SOUTH_HOUSTON', 'COP_HSL_SYSTEM_WIDE', 'COP_HSL_WEST_NORTH',
       'DELIVERY_DATE', 'DSTFlag', 'HOUR_BEGINNING', 'HOUR_ENDING',
       'STWPF_LZ_NORTH', 'STWPF_LZ_SOUTH_HOUSTON', 'STWPF_LZ_WEST',
       'STWPF_SOUTH_HOUSTON', 'STWPF_SYSTEM_WIDE', 'STWPF_WEST_NORTH',
       'WGRPP_LZ_NORTH', 'WGRPP_LZ_SOUTH_HOUSTON', 'WGRPP_LZ_WEST',
       'WGRPP_SOUTH_HOUSTON', 'WGRPP_SYSTEM_WIDE', 'WGRPP_WEST_NORTH'],
      dtype='object')

Seems like we actually care about actual and STWPF (short-term wind power forecast), less WGRPP (wind gen resource power potential) or COP HSL (current operating plan high sustainable limit) http://www.ercot.com/glossary/h

In [67]:
df_wind_sub = df_wind[['DELIVERY_DATE','HOUR_BEGINNING','HOUR_ENDING','ACTUAL_LZ_NORTH', 'ACTUAL_LZ_SOUTH_HOUSTON', 'ACTUAL_LZ_WEST',
       'ACTUAL_SOUTH_HOUSTON', 'ACTUAL_SYSTEM_WIDE', 'ACTUAL_WEST_NORTH','STWPF_LZ_NORTH', 'STWPF_LZ_SOUTH_HOUSTON', 'STWPF_LZ_WEST',
       'STWPF_SOUTH_HOUSTON', 'STWPF_SYSTEM_WIDE', 'STWPF_WEST_NORTH']]

In [70]:
df_wind_sub.shape

(5668213, 15)

In [71]:
df_wind_sub.head()

,DELIVERY_DATE,HOUR_BEGINNING,HOUR_ENDING,ACTUAL_LZ_NORTH,ACTUAL_LZ_SOUTH_HOUSTON,ACTUAL_LZ_WEST,ACTUAL_SOUTH_HOUSTON,ACTUAL_SYSTEM_WIDE,ACTUAL_WEST_NORTH,STWPF_LZ_NORTH,STWPF_LZ_SOUTH_HOUSTON,STWPF_LZ_WEST,STWPF_SOUTH_HOUSTON,STWPF_SYSTEM_WIDE,STWPF_WEST_NORTH
0,03/22/2018,NaN,5.0,686.95,2899.57,9815.77,NaN,13402.29,NaN,804.1,2768.1,11941.3,NaN,15513.5,NaN
1,03/22/2018,NaN,6.0,653.45,2776.28,10543.93,NaN,13973.66,NaN,798.0,2851.6,11866.8,NaN,15516.4,NaN
2,03/22/2018,NaN,7.0,671.39,2712.13,11113.10,NaN,14496.62,NaN,778.7,2747.0,11580.0,NaN,15105.7,NaN
3,03/22/2018,NaN,8.0,638.67,2643.54,11065.38,NaN,14347.59,NaN,753.8,2686.8,11309.0,NaN,14749.6,NaN
4,03/22/2018,NaN,9.0,706.22,2635.47,10526.52,NaN,13868.21,NaN,765.4,2746.9,10817.3,NaN,14329.6,NaN


In [72]:
df_wind_sub['hr_beg'] = df_wind_sub['HOUR_ENDING'] - 1

In [78]:
df_wind_sub.rename(columns={'DELIVERY_DATE':'date'}, inplace=True)

In [81]:
df_wind_sub['date'] = pd.to_datetime(df_wind_sub['date']).dt.date

In [85]:
df_wind_sub.drop(columns=['HOUR_BEGINNING','HOUR_ENDING'], inplace=True)

In [99]:
df_wind_sub.drop_duplicates(keep='first',inplace=True)

In [101]:
df_wind_sub.to_csv("df_wind.csv", index=False)

why is this dataframe SO LONG...oh--I bet it's because I assumed their headers were all the same, but they weren't...

In [106]:
hr = df_wind_sub['hr_beg']
df_wind_sub.drop(labels=['hr_beg'], axis=1,inplace = True)
df_wind_sub.insert(0, 'hr_beg', hr)
df_wind_sub.head()

,hr_beg,date,ACTUAL_LZ_NORTH,ACTUAL_LZ_SOUTH_HOUSTON,ACTUAL_LZ_WEST,ACTUAL_SOUTH_HOUSTON,ACTUAL_SYSTEM_WIDE,ACTUAL_WEST_NORTH,STWPF_LZ_NORTH,STWPF_LZ_SOUTH_HOUSTON,STWPF_LZ_WEST,STWPF_SOUTH_HOUSTON,STWPF_SYSTEM_WIDE,STWPF_WEST_NORTH
0,4.0,2018-03-22,686.95,2899.57,9815.77,NaN,13402.29,NaN,804.1,2768.1,11941.3,NaN,15513.5,NaN
1,5.0,2018-03-22,653.45,2776.28,10543.93,NaN,13973.66,NaN,798.0,2851.6,11866.8,NaN,15516.4,NaN
2,6.0,2018-03-22,671.39,2712.13,11113.10,NaN,14496.62,NaN,778.7,2747.0,11580.0,NaN,15105.7,NaN
3,7.0,2018-03-22,638.67,2643.54,11065.38,NaN,14347.59,NaN,753.8,2686.8,11309.0,NaN,14749.6,NaN
4,8.0,2018-03-22,706.22,2635.47,10526.52,NaN,13868.21,NaN,765.4,2746.9,10817.3,NaN,14329.6,NaN


In [107]:
df_wind_sub[df_wind_sub.duplicated(subset=['date','hr_beg'],keep='first')]

,hr_beg,date,ACTUAL_LZ_NORTH,ACTUAL_LZ_SOUTH_HOUSTON,ACTUAL_LZ_WEST,ACTUAL_SOUTH_HOUSTON,ACTUAL_SYSTEM_WIDE,ACTUAL_WEST_NORTH,STWPF_LZ_NORTH,STWPF_LZ_SOUTH_HOUSTON,STWPF_LZ_WEST,STWPF_SOUTH_HOUSTON,STWPF_SYSTEM_WIDE,STWPF_WEST_NORTH
217,NaN,NaN,NaN,NaN,NaN,1123.70,8252.44,7128.74,NaN,NaN,NaN,973.0,8372.8,7399.8
218,NaN,NaN,NaN,NaN,NaN,1173.37,8384.85,7211.48,NaN,NaN,NaN,1140.4,8385.0,7244.6
219,NaN,NaN,NaN,NaN,NaN,1250.15,8749.86,7499.71,NaN,NaN,NaN,1126.0,8344.2,7218.2
220,NaN,NaN,NaN,NaN,NaN,1079.14,8769.15,7690.01,NaN,NaN,NaN,1190.9,8547.9,7357.0
221,NaN,NaN,NaN,NaN,NaN,1017.59,7841.90,6824.31,NaN,NaN,NaN,1046.4,8305.8,7259.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5668208,3.0,2016-08-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,697.0,8115.2,7418.2
5668209,4.0,2016-08-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,574.8,7691.6,7116.8
5668210,5.0,2016-08-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,475.0,7280.6,6805.6
5668211,6.0,2016-08-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,395.6,6902.9,6507.3
